# Nucleotide diversity

## Using REF as ancestral alleles

We are wondering how the ancestral allele inference effect the final treesequence object.
We are testing a *tstree* object created by imposing the reference allele as the 
ancestral allele. The effect is that there are more or less the same mutations as the
number of variants (while in the case of the ancestrall allele calculated with 
`est-sfs` we can see million of mutations). Let's start by loading data for this
*REF-based* treeseq object

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tskit

from tskitetude import get_project_dir
from tskitetude.helper import create_windows

In [ ]:
ts_reference = tskit.load(str(get_project_dir() / "results-reference/background_samples/tsinfer/SMARTER-OA-OAR3-forward-0.4.10.focal.26.trees"))
ts_reference

I need to calculate nucleotide diversity *per site*. The only way to do this seems
to be calculating windows containing the SNPs an then calculating the nucleotide
diversity with the `tskit.TreeSequence.diversity` function. I've created a function
`create_windows` in `helper` module:

In [ ]:
# the last index is a simply a 2 step starting from position 1
reference_diversity = ts_reference.diversity(windows=create_windows(ts_reference))[1::2]
reference_diversity[:10]

Now let's compare the nucleotide diversity calculated using vcftools: here's the 
command line to calculate nucleotide diversity *per site*:

```bash
cd results-reference/background_samples/focal
vcftools --gzvcf SMARTER-OA-OAR3-forward-0.4.10.focal.26.vcf.gz --out allsamples_pi --site-pi
```

The `allsamples_pi.sites.pi` is a *TSV* file with the positions and the nucleotide diversity. Read it with pandas:

In [ ]:
vcftools_diversity = pd.read_csv(get_project_dir() / "results-reference/background_samples/focal/allsamples_pi.sites.pi", sep="\t")
vcftools_diversity.head()

Are this values similar?

In [ ]:
np.isclose(reference_diversity, vcftools_diversity["PI"], atol=1e-6).all()

Calculate diversity using *branch*:

In [ ]:
# the last index is a simply a 2 step starting from position 1
reference_diversity_branch = ts_reference.diversity(mode='branch', windows=create_windows(ts_reference))[1::2]
reference_diversity_branch[:10]

Try to plot the tow different diversities with vcftools output:

In [ ]:
plt.scatter(reference_diversity, vcftools_diversity["PI"])

In [ ]:
plt.scatter(reference_diversity_branch, vcftools_diversity["PI"])
plt.xlim(0, 1000)

## EST-SFS output as ancestral alleles

Can we calculate nucleotide diversity using the *tree files* generated by the pipeline
using the `est-sfs` output as ancestral alleles?

In [ ]:
ts_estsfs = tskit.load(str(get_project_dir() / "results-estsfs/background_samples/tsinfer/SMARTER-OA-OAR3-forward-0.4.10.focal.26.trees"))
ts_estsfs

Same stuff as before

In [ ]:
# the last index is a simply a 2 step starting from position 1
estsfs_diversity = ts_estsfs.diversity(windows=create_windows(ts_estsfs))[1::2]
estsfs_diversity[:10]

Are this values similar to the values calculated using VCFtools?

In [ ]:
np.isclose(estsfs_diversity, vcftools_diversity["PI"], atol=1e-6).all()

So *nucleotide diversity* is the same in both cases (using the REF as ancestral allele and using the `est-sfs` output as ancestral allele). Calculate diversity using *branch*:

In [ ]:
# the last index is a simply a 2 step starting from position 1
estsfs_diversity_branch = ts_estsfs.diversity(mode='branch', windows=create_windows(ts_estsfs))[1::2]
estsfs_diversity_branch[:10]

Try to plot the tow different diversities with vcftools output:

In [ ]:
plt.scatter(estsfs_diversity, vcftools_diversity["PI"])

In [ ]:
plt.scatter(estsfs_diversity_branch, vcftools_diversity["PI"])
plt.xlim(0, 1000)

## Compara output as ancestral alleles

Calculate nucleotide diversity using the *tree files* generated by the pipeline
using the `compara` approach as ancestral alleles.

In [ ]:
ts_compara = tskit.load(str(get_project_dir() / "results-compara/background_samples/tsinfer/SMARTER-OA-OAR3-forward-0.4.10.focal.26.trees"))
ts_compara

Calculate nucleotide diversity as before:

In [ ]:
# the last index is a simply a 2 step starting from position 1
compara_diversity = ts_compara.diversity(windows=create_windows(ts_compara))[1::2]
compara_diversity[:10]

Are this values similar to the values calculated using VCFtools?

In [ ]:
np.isclose(compara_diversity, vcftools_diversity["PI"], atol=1e-6).all()

So *nucleotide diversity* is the same in both cases (using the REF as ancestral allele and using the `compara` as ancestral allele). Calculate diversity using *branch*:

In [ ]:
# the last index is a simply a 2 step starting from position 1
compara_diversity_branch = ts_compara.diversity(mode='branch', windows=create_windows(ts_compara))[1::2]
compara_diversity_branch[:10]

Try to plot the tow different diversities with vcftools output:

In [ ]:
plt.scatter(compara_diversity, vcftools_diversity["PI"])

In [ ]:
plt.scatter(compara_diversity_branch, vcftools_diversity["PI"])
plt.xlim(0, 1000)